#### Document Re-ranking

In [60]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy.spatial.distance import cosine as cs
from numpy.linalg import norm

size ----> Dimension of the model embedding 

In [22]:
def sent_vectorizer(sent, model ,size):
    
    sent_vec = np.zeros(size)
    numw = 0
    for w in sent:
        try:
            sent_vec = np.add(sent_vec, model[w])
            numw+=1
        except:
            pass
    return sent_vec / np.sqrt(sent_vec.dot(sent_vec))

In [30]:
def get_query_embedding(query,model ,size):
    
    #Tokenization
    query_tok = query.split(' ')
    
    #Need to use the tokenization of the appropriate model
    
    #Extracting the query embedding
    query_embed = sent_vectorizer(query_tok, model ,size)
    
    return query_embed
    

In [53]:
def get_ans_embeddings(list_ans,model ,size):
    
    #Tokenization ---> List Comprehension
    list_ans = [ans.split(' ') for ans in list_ans]
    
    #Getting embedding ----> List comprehensions
    ans_embed = [sent_vectorizer(ans_tok,model,size) for ans_tok in list_ans]
    
    #Need to use the tokenization of the appropriate model
    return np.array(ans_embed)
    

In [150]:
def get_cosine_sim(query, list_ans, model ,size,topk) :
    
    #Input
    question = query
    answers = list_ans
    
    
    #Query embedding
    query_embed = get_query_embedding(query,model ,size)
    query_embed_norm = query_embed/norm(query_embed)
    
    #Answers embedding
    ans_embed = get_ans_embeddings(list_ans,model ,size)
    ans_embed_norm = ans_embed/norm(ans_embed,axis=1,keepdims= True)
    
    cos_sim = np.dot(ans_embed_norm,query_embed_norm)
    
    
    #Output Format
    df_out = pd.DataFrame()
    df_out['answers'] = answers
    df_out['similarity'] = cos_sim
    df_out['Query'] = question
    
    #Output sorted
    final_df = df_out.sort_values(by=['similarity'], ascending=False)[['Query','answers','similarity']]

    
    return final_df.head(topk)

In [ ]:
final_df = get_cosine_sim(query, list_ans, model ,size,topk)